<b><h1 align="center">Proceso ETL</h1></b>
<b><h1 align="center">Extracción, Transformación y Carga de Datos</h1></b>
</i>



---

Antes de pasar a analizar los datos y crear algoritmos predictivos, debemos en primer lugar capturar los datos, transformarlos adecuadamente para evitar valores inadecuados y finalmente cargarlos en la base de datos que utilizaremos. Este proceso de extracción, transformación y carga (ETL por sus siglas en inglés) se automatiza en el presente documento.

Hay tres tipos básicos de datos que serán tratados:
- Datos de clinetes (provenientes del CRM)
- Datos de ventas (provenientes del CRM)
- Datos de variables macro economicas (Banco Central de Chile)

En todos los casos fue necesario descargar los datos a CSV. Si bien el Banco Central de Chile tiene su propia API, esta solo está disponibles para ciertos usuarios.

## Importar bibliotecas necesarias

In [69]:
import pandas as pd
import numpy as np

# CLIENTES

## Cargar datos de clientes
Cambiar la dirección del archivo siguiente según sea necesario

In [70]:
clientes = pd.read_csv("Data/clientes.csv")

clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87490 entries, 0 to 87489
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rut Cliente            64442 non-null  object 
 1   Nombre Cliente         87486 non-null  object 
 2   Nivel Soporte          34360 non-null  object 
 3   Area Cliente           67691 non-null  object 
 4   Sector Cliente         42777 non-null  object 
 5   Tipo Cliente           87486 non-null  object 
 6   Industria Cliente      83255 non-null  object 
 7   Id Cliente             87482 non-null  float64
 8   País Cliente           87486 non-null  object 
 9   Comuna Cliente         80784 non-null  object 
 10  Ciudad Cliente         77374 non-null  object 
 11  Cargo Contacto         49136 non-null  object 
 12  Función Contacto       46971 non-null  object 
 13  Clasificación Cliente  87452 non-null  object 
 14  Categoría 3D           54706 non-null  object 
 15  Di

## Limpiar datos de clientes

In [71]:
# Número de valores null por columna
clientes.isnull().sum()

Rut Cliente              23048
Nombre Cliente               4
Nivel Soporte            53130
Area Cliente             19799
Sector Cliente           44713
Tipo Cliente                 4
Industria Cliente         4235
Id Cliente                   8
País Cliente                 4
Comuna Cliente            6706
Ciudad Cliente           10116
Cargo Contacto           38354
Función Contacto         40519
Clasificación Cliente       38
Categoría 3D             32784
Dicom                    82968
Productos Cliente        54057
Software Clientes        87488
Contacto Activo          10018
Industria Contacto       17541
Asociación Cliente       73084
Column1                  87490
dtype: int64

Existen muchos valores null. Es preferible prescindir de las columnas con un alto porcentaje de nulls y mantener únicamente las columnas con valores aceptables.

Además, se ejecuta las siguientes operaciones:

1.   Orderar las columnas, haciendo que el ID sea la primera
2.   Eliminar filas donde sigan existiendo datos null
3.   Cambiar el tipo de dato en la columna ID a "int"

In [72]:
mantener_columnas = ["Id Cliente",
                     "Nombre Cliente",
                     "Tipo Cliente",
                     "País Cliente",
                     "Clasificación Cliente"]
clientes = clientes[mantener_columnas].copy()
clientes = clientes.dropna()
clientes["Id Cliente"] = clientes["Id Cliente"].astype(int)
clientes.head()

,Id Cliente,Nombre Cliente,Tipo Cliente,País Cliente,Clasificación Cliente
0,34613,SEBASTIAN ANDRES YEVENES RODRIGUEZ,Activo,Chile,BRONCE
1,35070,UNIVERSIDAD DE CHILE,Activo,Chile,PLATINIUM
2,35123,UNIVERSIDAD DE CHILE,Activo,Chile,SILVER
3,35270,UNIVERSIDAD DE CHILE,Activo,Chile,SILVER
4,35808,SANTIAGO BECKDORF SOLA,Activo,Chile,BRONCE


In [73]:
# Valores posibles en la columna "Tipo Cliente"
print(clientes["Tipo Cliente"].unique())

['Activo' 'Nuevo/Existente' 'Cliente competencia']


En la tabla de clientes existe una columna llamada "Tipo Cliente" con tres posibles valores:


1.   Activo
2.   Nuevo/Existente
3.   Cliente competencia

## Guardar datos procesados

Utilizaremos HDFS (Hadoop Distributed File System) para crear una base de datos procesados. Esto facilitará la escalabilidad de uso de los datos a medida que su volumen aumente.


In [74]:
clientes.to_hdf("Data/data.h5", key="clientes")

Solo para que quede como referencia, se deja a continuación el código necesario para cargar los datos desde el archivo .h5. 
En ocasiones puede ser necesario actualizar el paquete "tables" para que sea compatible con la versión de Pandas en uso.

In [75]:
#!pip install --upgrade tables
#cliente_loaded = pd.read_hdf("Data/data.h5", key="clientes", mode="r")

# VENTAS

## Cargar datos de ventas

Los datos de ventas se encuentran dispersos en archivos de ventas anuales. El hitórico va desde 2010 hasta 2021. El primer paso en este caso es cargar los datos históricos en un único DataFrame.

In [76]:
folder = "Data/"

df_list = [pd.read_csv(folder + f"ventas{year}.csv", 
                       index_col=None, 
                       header=0,
                       thousands=",",
                       dtype={'Origen solicitud': 'str', 'Observaciones': 'str'}) 
            for year in range(2010, 2022)]

ventas = pd.concat(df_list, axis=0, ignore_index=True)

ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188826 entries, 0 to 188825
Data columns (total 34 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Factura Venta           188809 non-null  float64
 1   Fec. Fact. Venta        188809 non-null  object 
 2   Cod. Producto           188809 non-null  object 
 3   Unnamed: 3              0 non-null       float64
 4   Descripción             188809 non-null  object 
 5   Grupo                   188760 non-null  object 
 6   Unnamed: 6              0 non-null       float64
 7   Subgrupo                188766 non-null  object 
 8   Serie                   148952 non-null  object 
 9   Cantidad                188809 non-null  float64
 10  Valor                   188809 non-null  float64
 11  Costo                   188809 non-null  float64
 12  Margen                  188809 non-null  float64
 13  Costo Manual            188809 non-null  float64
 14  Recargo IVA         

In [77]:
ventas.head()

,Factura Venta,Fec. Fact. Venta,Cod. Producto,Unnamed: 3,Descripción,Grupo,Unnamed: 6,Subgrupo,Serie,Cantidad,...,Proveedor,OC,Nota Venta,Descripción Nota Venta,Región (Softland),Comuna (Softland),Región (CRM),Comuna (CRM),Observaciones,Se calculó el costo
0,10.0,28-07-2010,LEC,NaN,DONDE DICE,GASTOS,NaN,GASTOS,NaN,1.0,...,METALURGICA SILCOSIL LTDA.,0.0,102354.0,SERV. CONSULTORIA CORRESPONDIENTE A 150 USD,NaN,NaN,NaN,NaN,NaN,Si
1,29001.0,08-01-2010,CE528A,NaN,IMPRESORA HP LASERJET P315 DN,IMPRESORAS HP,NaN,IMPRESORAS LASER HP,SBRBH9C4N28,1.0,...,TECNOGLOBAL S.A.,9047.0,100989.0,NaN,8.0,LOS ANGELES,Octava Región (del Bío-Bío),Los Angeles,NaN,Si
2,29002.0,08-01-2010,GASTDESP,NaN,GASTOS DE DESPACHO,GASTOS,NaN,GASTOS,NaN,1.0,...,MINERA SANTO DOMINGO SCM,0.0,100980.0,NaN,13.0,HUECHURABA,Región Metropolitana (de Santiago),Huechuraba,NaN,Si
3,29002.0,08-01-2010,BOND90X50-80,NaN,"ROLLO PAPEL BOND 0,914 X 50 MTS 80 GR",SUSTRATOS,NaN,SUSTRATOS GENERICOS,NaN,5.0,...,NEXSYS DE CHILE S A,15276.0,100980.0,NaN,13.0,HUECHURABA,Región Metropolitana (de Santiago),Huechuraba,NaN,Si
4,29002.0,08-01-2010,BOND90X50-80,NaN,"ROLLO PAPEL BOND 0,914 X 50 MTS 80 GR",SUSTRATOS,NaN,SUSTRATOS GENERICOS,NaN,5.0,...,DISTRIBUIDORA DIAZOL S.A.,9045.0,100980.0,NaN,13.0,HUECHURABA,Región Metropolitana (de Santiago),Huechuraba,NaN,Si


## Limpiar datos de ventas

In [78]:
# Número de valores null por columna
null_count_ventas = ventas.isnull().sum()
null_count_ventas

Factura Venta                 17
Fec. Fact. Venta              17
Cod. Producto                 17
Unnamed: 3                188826
Descripción                   17
Grupo                         66
Unnamed: 6                188826
Subgrupo                      60
Serie                      39874
Cantidad                      17
Valor                         17
Costo                         17
Margen                        17
Costo Manual                  17
Recargo IVA                   17
Rut                           46
Cliente                       17
Industria                   2167
Vendedor                    1067
Tipo Vendedor               1320
Origen solicitud          188823
Fact. Compra                4498
Fec. Fact. Compra           2520
Fec. Ing. Fact. Compra      2520
Proveedor                   2520
OC                          2520
Nota Venta                    17
Descripción Nota Venta     49663
Región (Softland)            810
Comuna (Softland)            242
Región (CR

De la operación anterior podemos ver que existen campos con una gran cantidad de valores nulos. Aprovechándonos de este conteo vamos a establecer el criterio de prescindir de las columnas cuyo conteo de valores nulos superen los 10000 valores.

In [79]:
columns = null_count_ventas.index
new_cols = [column for column in columns if null_count_ventas[column] <= 10000]
ventas = ventas[new_cols]
ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188826 entries, 0 to 188825
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Factura Venta           188809 non-null  float64
 1   Fec. Fact. Venta        188809 non-null  object 
 2   Cod. Producto           188809 non-null  object 
 3   Descripción             188809 non-null  object 
 4   Grupo                   188760 non-null  object 
 5   Subgrupo                188766 non-null  object 
 6   Cantidad                188809 non-null  float64
 7   Valor                   188809 non-null  float64
 8   Costo                   188809 non-null  float64
 9   Margen                  188809 non-null  float64
 10  Costo Manual            188809 non-null  float64
 11  Recargo IVA             188809 non-null  float64
 12  Rut                     188780 non-null  object 
 13  Cliente                 188809 non-null  object 
 14  Industria           

Análisando los valores, las columnas restantes con algún valor null se pueden completar los datos como sigue:


1.   En la columna industria, se puede rellenar con "GENERAL" los valores nan
2.   En la columna Vendedor se puede rellanar con "NO APLICA" los valores nan
3.   Columnas "Grupo" y "Subgrupo" -> "SIN GRUPO"
4.   Columnas "Cantidad", "Valor", "Costo", "Margen", "Costo Manual", "Recargo IVA" -> 0
5.   En el caso de las fechas, rellenaremos los valores nulos utilizando el método "Forward Fill".
6.   Se eliminaran las líneas donde el valor de "Factura Venta" sea null.
6.   El resto de los valores null, se completará con el valor "-1". De tal manera que en procesos futuros que impliquen el uso de los datos, se puedan filtrar estos valores a discreción.




In [80]:
def rellenar_nulls(replacement):
    """Rellenar valores null con valor especificado
    Args:
    values: diccionario con Keys representando el título de la columna
        y Values representando el valor que sustituira null
    Return:
    None
    """
    for col in replacement.keys():
        ventas[col].fillna(value=replacement[col], inplace=True)

In [81]:
replacement = {"Industria": "GENERAL",
               "Vendedor": "NO APLICA",
               "Grupo": "SIN GRUPO",
               "Subgrupo": "SIN GRUPO",
               "Cantidad": 0,
               "Valor": 0,
               "Costo": 0,
               "Margen": 0,
               "Costo Manual": 0,
               "Recargo IVA": 0}

rellenar_nulls(replacement)

## Transformación de datos

Se convierten los valores de fecha a Datetime y los valores enteros a int.

In [82]:
# Rellenar fecha con valores anteriores
fechas = ["Fec. Fact. Venta", "Fec. Fact. Compra", "Fec. Ing. Fact. Compra"]

for fecha in fechas:
    ## Primero debemos convertir los valores string a formato Datetime
    ventas[fecha] = pd.to_datetime(ventas[fecha], format="%d-%m-%Y")
    ## Luego vamos a rellenar la fechas faltantes con el valor anterior
    ventas[fecha].fillna(method="ffill", inplace=True)


# Eliminar filas donde no exista valor de "Factura Venta"
ventas = ventas.drop(ventas[ventas["Factura Venta"].isnull()].index)

# El resto de los valores null los rellenamos con "-1"
ventas.fillna(value="-1", inplace=True)

In [83]:
# Pasar a entero valores de ID
to_integer = ["Factura Venta", "Fact. Compra", "OC", "Nota Venta", "Región (Softland)"]
for column in to_integer:
    ventas[column] = ventas[column].astype(np.int64)

print(ventas.dtypes)

Factura Venta                      int64
Fec. Fact. Venta          datetime64[ns]
Cod. Producto                     object
Descripción                       object
Grupo                             object
Subgrupo                          object
Cantidad                         float64
Valor                            float64
Costo                            float64
Margen                           float64
Costo Manual                     float64
Recargo IVA                      float64
Rut                               object
Cliente                           object
Industria                         object
Vendedor                          object
Tipo Vendedor                     object
Fact. Compra                       int64
Fec. Fact. Compra         datetime64[ns]
Fec. Ing. Fact. Compra    datetime64[ns]
Proveedor                         object
OC                                 int64
Nota Venta                         int64
Región (Softland)                  int64
Comuna (Softland

In [84]:
# Número de valores null por columna
# Solo para verificar que hemos tratado todos los datos null
null_count_ventas = ventas.isnull().sum()
null_count_ventas

Factura Venta             0
Fec. Fact. Venta          0
Cod. Producto             0
Descripción               0
Grupo                     0
Subgrupo                  0
Cantidad                  0
Valor                     0
Costo                     0
Margen                    0
Costo Manual              0
Recargo IVA               0
Rut                       0
Cliente                   0
Industria                 0
Vendedor                  0
Tipo Vendedor             0
Fact. Compra              0
Fec. Fact. Compra         0
Fec. Ing. Fact. Compra    0
Proveedor                 0
OC                        0
Nota Venta                0
Región (Softland)         0
Comuna (Softland)         0
Región (CRM)              0
Comuna (CRM)              0
Se calculó el costo       0
dtype: int64

Finalmente vamos a orderan lo valores según su fecha de venta.

In [85]:
ventas.sort_values(by=["Fec. Fact. Venta"], inplace=True)
ventas.head()

,Factura Venta,Fec. Fact. Venta,Cod. Producto,Descripción,Grupo,Subgrupo,Cantidad,Valor,Costo,Margen,...,Fec. Fact. Compra,Fec. Ing. Fact. Compra,Proveedor,OC,Nota Venta,Región (Softland),Comuna (Softland),Región (CRM),Comuna (CRM),Se calculó el costo
23,29009,2010-01-08,C9373A,TINTA AMARILLA 130ML HP 72 C9373A PARA PLOTTER...,INSUMOS HP,INSUMOS TINTAS PLOTTER HP,1.0,35165.0,30072.0,5093.0,...,2010-01-02,2010-02-02,TECNOGLOBAL S.A.,9013,100975,13,PROVIDENCIA,Región Metropolitana (de Santiago),Providencia,Si
31,29013,2010-01-08,BOND60X50-80,ROLLO PAPEL BOND 60 X 50 MTS 80 GR,SUSTRATOS,SUSTRATOS GENERICOS,5.0,16000.0,12000.0,4000.0,...,2009-12-22,2009-12-23,DISTRIBUIDORA DIAZOL S.A.,9000,100994,13,SANTIAGO,Región Metropolitana (de Santiago),Santiago,Si
30,29012,2010-01-08,CH337A,"PLOTTER HP DESIGNJET 510 PRINTER 42""",PLOTTER,PLOTTER CAD HP,1.0,1604948.0,1364136.0,240812.0,...,2010-01-07,2010-01-08,INGRAM MICRO CHILE S.A.,9016,100990,13,LAS CONDES,Región Metropolitana (de Santiago),Vitacura,Si
29,29012,2010-01-08,BOND107X50-80,"ROLLO PAPEL BOND 1,07 X 50 MTS 80 GR",SUSTRATOS,SUSTRATOS GENERICOS,1.0,1.0,4125.0,-4124.0,...,2009-11-17,2009-11-18,DISTRIBUIDORA DIAZOL S.A.,8914,100990,13,LAS CONDES,Región Metropolitana (de Santiago),Vitacura,Si
28,29012,2010-01-08,044FCJ,CABLE PRINTER USB AB 1.6 MTS,HARDWARE,HARDWARE OTROS,1.0,1.0,759.0,-758.0,...,2008-12-05,2008-12-09,INGRAM MICRO CHILE S.A.,7940,100990,13,LAS CONDES,Región Metropolitana (de Santiago),Vitacura,Si


## Guardar datos procesados

Utilizaremos HDFS (Hadoop Distributed File System) para crear una base de datos procesados. Esto facilitará la escalabilidad de uso de los datos a medida que su volumen aumente.

In [86]:
ventas.to_hdf("Data/data.h5", key="ventas", mode="w")

# VARIABLES MACROECONÓMICAS

## Cargar datos de variables macroeconómicas
Los datos fueron tomados de la pagina oficial del Banco Central de Chile y cada indicador se encuentra en un archivo CSV por separado.

Antes de pasar a la unificación de estos datos en un único DataFrame, crearemos un rango de fechas para luego pasar los valores por un join. Empezaremos desde enero de 2010 hasta la fecha.

In [87]:
import datetime

start = datetime.datetime.strptime("01-01-2010", "%d-%m-%Y")
end = datetime.datetime.strptime("03-02-2022", "%d-%m-%Y")
date_range = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
date_range = pd.DataFrame(data=np.array(date_range), columns=["Fecha"])

Cargemos ahora cada tabla de datos macroeconómicos

In [88]:
chp_usd = pd.read_csv("Data/CHPUSD.csv")
ipc = pd.read_csv("Data/IPC.csv")
imacec = pd.read_csv("Data/IMACEC.csv")
pib = pd.read_csv("Data/PIB.csv")
tcm = pd.read_csv("Data/TCM.csv")
tdn = pd.read_csv("Data/TDN.csv")
uf = pd.read_csv("Data/UF.csv")

## Transformación de datos

Ahora debemos formatear adecuadamente las fechas que se encuentra en formato de cadena de texto. A fin de evitar problemas al pasar el formato a Datetime, vamos a agregar ceros a la izquierda de los meses y día cuando tengan un solo dígito.

In [89]:
# Función para agregar ceros a la izquierda en mes y día
def zero_padded_date(date):
    split = date.split("/")
    if len(split[0]) < 2:
        split[0] = "0" + split[0]
    if len(split[1]) < 2:
        split[1] = "0" + split[1]
    return f"{split[0]}/{split[1]}/{split[2]}"

Vamos a hacer las conversiones necesarias para cada tabla previamente cargada.

In [90]:
chp_usd["Dia"] = chp_usd["Dia"].apply(zero_padded_date)
chp_usd["Fecha"] = pd.to_datetime(chp_usd["Dia"], format="%m/%d/%Y")
chp_usd = chp_usd[["Fecha", "Dólar"]]
chp_usd.head(1)

,Fecha,Dólar
0,2010-01-04,507.1


In [91]:
ipc["Fecha"] = ipc["Fecha"].apply(zero_padded_date)
ipc["Fecha"] = pd.to_datetime(ipc["Fecha"], format="%m/%d/%Y")
ipc = ipc[["Fecha", "IPC"]]
ipc.head(1)

,Fecha,IPC
0,1928-03-01,0.0


In [92]:
imacec["Fecha"] = imacec["Fecha"].apply(zero_padded_date)
imacec["Fecha"] = pd.to_datetime(imacec["Fecha"], format="%m/%d/%Y")
imacec = imacec[["Fecha", "IMACEC"]]
imacec.head(1)

,Fecha,IMACEC
0,2010-01-01,2.0


In [93]:
pib["Fecha"] = pib["Fecha"].apply(zero_padded_date)
pib["Fecha"] = pd.to_datetime(pib["Fecha"], format="%m/%d/%Y")
pib = pib[["Fecha", "PIB (MM)"]]
pib.head(1)

,Fecha,PIB (MM)
0,2010-03-01,"27,890"


In [94]:
tcm["Fecha"] = tcm["Fecha"].apply(zero_padded_date)
tcm["Fecha"] = pd.to_datetime(tcm["Fecha"], format="%m/%d/%Y")
tcm = tcm[["Fecha", "TCM"]]
tcm.head(1)

,Fecha,TCM
0,2010-01-04,103.41


In [95]:
tdn["Fecha"] = tdn["Fecha"].apply(zero_padded_date)
tdn["Fecha"] = pd.to_datetime(tdn["Fecha"], format="%m/%d/%Y")
tdn = tdn[["Fecha", "TDN"]]
tdn.head(1)

,Fecha,TDN
0,2010-03-01,9.23


In [96]:
uf["Fecha"] = uf["Fecha"].apply(zero_padded_date)
uf["Fecha"] = pd.to_datetime(uf["Fecha"], format="%m/%d/%Y")
uf = uf[["Fecha", "UF"]]
uf.head(1)

,Fecha,UF
0,2010-01-01,"20,939.49"


A continuación, pasaremos a unir todas las columnas en un único DataFrame utilizando como valor clave la fecha. Cuidando además de no eliminar ninguna fila en el proceso.

In [97]:
vme = date_range.set_index("Fecha").join(chp_usd.set_index("Fecha"), on="Fecha", how="left")
vme = vme.join(ipc.set_index("Fecha"), on="Fecha", how="left")
vme = vme.join(imacec.set_index("Fecha"), on="Fecha", how="left")
vme = vme.join(pib.set_index("Fecha"), on="Fecha", how="left")
vme = vme.join(tcm.set_index("Fecha"), on="Fecha", how="left")
vme = vme.join(tdn.set_index("Fecha"), on="Fecha", how="left")
vme = vme.join(uf.set_index("Fecha"), on="Fecha", how="left")
vme = vme.fillna(method="ffill").fillna(method="bfill")
vme.head()

,Dólar,IPC,IMACEC,PIB (MM),TCM,TDN,UF
Fecha,,,,,,,
2010-01-01,507.1,0.5,2.0,"27,890",103.41,9.23,"20,939.49"
2010-01-02,507.1,0.5,2.0,"27,890",103.41,9.23,"20,936.11"
2010-01-03,507.1,0.5,2.0,"27,890",103.41,9.23,"20,932.72"
2010-01-04,507.1,0.5,2.0,"27,890",103.41,9.23,"20,929.34"
2010-01-05,505.7,0.5,2.0,"27,890",103.56,9.23,"20,925.95"


Ahora comprobemos el tipo de datos resultantes y si existen o no valores null.

In [98]:
vme.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4416 entries, 2010-01-01 to 2022-02-02
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Dólar     4416 non-null   float64
 1   IPC       4416 non-null   float64
 2   IMACEC    4416 non-null   float64
 3   PIB (MM)  4416 non-null   object 
 4   TCM       4416 non-null   float64
 5   TDN       4416 non-null   float64
 6   UF        4416 non-null   object 
dtypes: float64(5), object(2)
memory usage: 276.0+ KB


Podemos observar que dos de las columnas tiene "object" como tipo de dato. Aunque por su valores podemos asumir que se trata de datos numéricos. Vamos a hacer la conversión oportuna de estos tipos de datos.

In [99]:
vme["PIB (MM)"] = vme["PIB (MM)"].str.replace(",", "").astype(float)
vme["UF"] = vme["UF"].str.replace(",", "").astype(float)

Finalmente, haremos algunos ajustes para unificar los títulos de las columnas. Evitaremos utilizar caracteres especiales, tales como tildes. Y evitaremos colocar entre paréntesis las unidades.

In [100]:
vme.rename(columns={"Dólar": "USDCHP", "PIB (MM)": "PIB"}, inplace=True)
vme.head(1)

,USDCHP,IPC,IMACEC,PIB,TCM,TDN,UF
Fecha,,,,,,,
2010-01-01,507.1,0.5,2.0,27890.0,103.41,9.23,20939.49


## Guardar datos procesados

Utilizaremos HDFS (Hadoop Distributed File System) para crear una base de datos procesados. Esto facilitará la escalabilidad de uso de los datos a medida que su volumen aumente.


In [101]:
vme.to_hdf("Data/data.h5", key="vme")